<h1>| pricker-v.0.1.36 |</h1>
<h3>Price validation tool for bloomberg-request-software format prices</h3>

In [1]:
###pricker-v.0.1.3 author A.SOLBIATI 07/07/17###
import os
import pandas
import datetime
from pandas_datareader import data as web
from bs4 import BeautifulSoup
import Tkinter
import logging
logging.basicConfig(filename='pricker-errors.log', level=logging.ERROR)


In [2]:
def read_file(FNAME, FIXED_LINES):
    'read input file skipping the first FIXED_LINES lines'
    with open(FNAME) as f:
        lines = f.readlines()
    lines = [x.strip() for x in lines] 
    lines = lines[FIXED_LINES:]
    
    return lines

In [3]:
CONFIG = os.path.abspath("config.txt")
config_lines = read_file((CONFIG),10)

#FNAME: path for input file (bloomberg-request-software format)
READFROMCONFIG = int(config_lines[0])
if (READFROMCONFIG==1):
    #FNAME = config_lines[1][config_lines[0].index(' ')+1:]
    FNAME = config_lines[1]
    FIXED_LINES = config_lines[2]
    FILENAME_INPUT= config_lines[3]
    FILENAME_OUTPUT= config_lines[4]
    MARKET = config_lines[5]
    
else:
    FNAME = os.path.abspath("Prezzi"+datetime.datetime.today().strftime('%d%m%y')+".out"+datetime.datetime.today().strftime('%Y%m%d'))
    #FIXED_LINES:  number of fixed lines to be skipped in input file
    FIXED_LINES = 30
    #MARKET: market to run the pricker on {'EQUITITES US', 'EQUITIES IM'}
    MARKET = "EQUITIES US"
    FILENAME_INPUT= os.path.abspath("www/template.html")
    FILENAME_OUTPUT= os.path.abspath("www/report.html")



In [4]:
CONFIG, READFROMCONFIG, FNAME, FIXED_LINES, FILENAME_INPUT, FILENAME_OUTPUT, MARKET

('/Users/alex/Desktop/Coding/BERT/pricker/config.txt',
 0,
 '/Users/alex/Desktop/Coding/BERT/pricker/Prezzi100817.out20170810',
 30,
 '/Users/alex/Desktop/Coding/BERT/pricker/www/template.html',
 '/Users/alex/Desktop/Coding/BERT/pricker/www/report.html',
 'EQUITIES US')

In [5]:
def checkfor(line, strs):
    #return 1 if there are all strs[i] in line
    _bool = True
    for _str in strs:
        if(line.find(_str)==-1):
            _bool = False
    return _bool

<h3><u>Equities</u> formatting:</h3>

In [6]:
def equity_market(line):
    return line[line.index(' ')+1:line.index(' ')+3]

def equity_name(line):
    return line[:line.index(' ')]

def equity_vals(line):
    vec = []
    for i in range(4,12):
        vec.append(line.split('|')[i])
    return vec

class Equity():
    
    def __init__(self, line):
        self.market=equity_market(line)
        self.name=equity_name(line)
        self.vals=equity_vals(line)
        if(self.vals[5]!='N.A.' and self.vals[5]!=''):
            self.close=float(self.vals[5])
        else:
            self.close=0
        self.curr=equity_vals(line)[7]
    
    def summary(self):
        print '\nEquity summary'
        print '- Name: ',self.name,' Market: ',self.market,' Close: ',self.close,' Curr: ',self.curr
        
    


In [7]:
def parse_equities(FNAME, FIXED_LINES):
	#parse equities from the input file populating the Equity object arrays
    
    lines = read_file(FNAME, FIXED_LINES)
    
    equities_US = []
    equities_IM = []
    
    for line in lines:
        if(checkfor(line,'Equity') and len(line.split(" "))<4):
            try:
                equity = Equity(line)
                if(equity.market=='US'):
                    equities_US.append(equity)
                if(equity.market=='IM'):
                    equities_IM.append(equity)
            except:
                logging.exception(datetime.datetime.now())
                
    
    return equities_US, equities_IM

In [8]:
def pricker_equities(equities, market="US", source="google"):
    #cross-validate the input prices with source prices using the Equity object arrays
    
    anomalies= []
    comparisons=0
    dataset=len(equities)
    
    #loop in the equities objects
    for equity in equities:
        
        try:
            #compare1: dowload data from source
            compare1 = float(web.DataReader(equity.name, source, datetime.datetime.today() - datetime.timedelta(days=1),datetime.datetime.today()).Close)
            
            #compare2: from the input file
            compare2 = equity.close
            
            if(compare1!=compare2):
                anomaly = equity.name,equity.close,compare1
                anomalies.append(anomaly)
                print len(anomalies),'FOUND ANOMALY (name,bloomberg,source)',anomaly
                
            else:
                print " | dowloaded [",compare1,"] from ",source,", regular [",compare2,"] on: ",equity.name
           
            comparisons=comparisons+1
            
        except Exception as e: print(e)
            
    print '\nANOMALIES:\n'
    for anomaly in anomalies:
        print anomaly
    print 'COMPARISONS:', comparisons,' (', dataset-comparisons, ' not found from source )'
    
    return anomalies, comparisons, dataset


In [9]:
def update_html(anomalies, comparisons, dataset, FILENAME_INPUT=os.path.abspath("template.html"), FILENAME_OUTPUT=os.path.abspath("report.html")):
    
    with open(FILENAME_INPUT) as fp:
        soup = BeautifulSoup(fp, "html5lib")
    
    for anomaly in anomalies:
        new_tag = soup.new_tag('li')
        
        new_tag = soup.new_tag('li')

        a = soup.new_tag('a')
        a_content = anomaly[0]+' '
        a.string=unicode(a_content)
        a['class']="small_a"

        p = soup.new_tag('p')
        p_content = 'Bloomberg:'+str(anomaly[1])+' Google: '+str(anomaly[2])
        p['style']="font-size:9px"
        p.string=unicode(p_content)
        a.insert(1,p)

        new_tag.append(a)
        
        soup.body.find('ul').append(new_tag)
    
    soup.find_all('a')[0].string = str(datetime.datetime.today())[:16]+" CHECK completed"
    
    soup.find_all('span')[0].string = "Regular: "+str(comparisons)
    soup.find_all('span')[1].string = "Anomalies: "+str(len(anomalies))
    soup.find_all('a')[1].string = "Instruments found from the source "+str(comparisons)+"/"+str(dataset)
    
    with open(FILENAME_OUTPUT, "wb") as f_output:
        f_output.write(soup.prettify("utf-8"))  

<h3>Procedures</h3>

In [10]:
try:
    equities_US, equities_IM = parse_equities(FNAME,int(FIXED_LINES))
except:
    logging.exception(datetime.datetime.now())
    
    
if(MARKET=='EQUITIES US'):
    
    try:
        anomalies, comparisons, dataset = pricker_equities(equities_US)
    except:
        logging.exception(datetime.datetime.now())
    
if(MARKET=='EQUITIES IM'):
    print '..not implemented yet'

1 FOUND ANOMALY (name,bloomberg,source) ('BTI', 0, 64.48)
2 FOUND ANOMALY (name,bloomberg,source) ('DRYS', 1.23, 1.77)
3 FOUND ANOMALY (name,bloomberg,source) ('NEM', 36.59, 36.1)
4 FOUND ANOMALY (name,bloomberg,source) ('CLR', 32.19, 34.31)
5 FOUND ANOMALY (name,bloomberg,source) ('AZN', 30.34, 29.47)
6 FOUND ANOMALY (name,bloomberg,source) ('GOLD', 95.1, 95.59)
7 FOUND ANOMALY (name,bloomberg,source) ('AAPL', 155.57, 161.06)
8 FOUND ANOMALY (name,bloomberg,source) ('ABBV', 70.99, 72.02)
Unable to read URL: http://www.google.com/finance/historical?q=ABT&startdate=Aug+09%2C+2017&enddate=Aug+10%2C+2017&output=csv
9 FOUND ANOMALY (name,bloomberg,source) ('ABX', 17.06, 16.73)
10 FOUND ANOMALY (name,bloomberg,source) ('ACHN', 3.9, 4.81)
11 FOUND ANOMALY (name,bloomberg,source) ('ACIA', 45.22, 46.25)
12 FOUND ANOMALY (name,bloomberg,source) ('ADSK', 107.88, 107.45)
Unable to read URL: http://www.google.com/finance/historical?q=AGN&startdate=Aug+09%2C+2017&enddate=Aug+10%2C+2017&output=csv
1

112 FOUND ANOMALY (name,bloomberg,source) ('MD', 44.37, 42.71)
Unable to read URL: http://www.google.com/finance/historical?q=MDIT&startdate=Aug+09%2C+2017&enddate=Aug+10%2C+2017&output=csv


In [5]:
try:
    equities_US, equities_IM = parse_equities(FNAME,int(FIXED_LINES))
    
    if(MARKET=='EQUITIES US'):        
        anomalies, comparisons, dataset = pricker_equities(equities_US)
    if(MARKET=='EQUITIES IM'):
        print '..not implemented yet'

    update_html(anomalies,comparisons,dataset,FILENAME_INPUT,FILENAME_OUTPUT)
    
except:
    logging.exception(datetime.datetime.now())

In [12]:
anomalies

[('BTI', 65.19, 64.48),
 ('DRYS', 1.23, 1.77),
 ('NEM', 36.59, 36.1),
 ('CLR', 32.19, 34.31),
 ('AZN', 30.34, 29.47),
 ('GOLD', 95.1, 95.59),
 ('AAPL', 155.57, 161.06),
 ('ABBV', 70.99, 72.02),
 ('ABX', 17.06, 16.73),
 ('ACHN', 3.9, 4.81),
 ('ACIA', 45.22, 46.25),
 ('ADSK', 107.88, 107.45),
 ('ALB', 118.8, 113.16),
 ('AMD', 13.24, 12.83),
 ('AMGN', 174.18, 173.55),
 ('AMZN', 986.92, 982.01),
 ('APC', 43.77, 44.1),
 ('AXP', 85.54, 85.69),
 ('BABA', 153.32, 157.49),
 ('BAC', 24.37, 24.74),
 ('BDX', 198.9, 197.84),
 ('BMY', 55.5, 57.4),
 ('BRK/B', 178.99, 178.06),
 ('BSMX', 10.49, 10.41),
 ('BWA', 45.88, 46.53),
 ('C', 68.13, 68.43),
 ('CAT', 113.56, 114.34),
 ('CBOE', 94.99, 96.8),
 ('CBS/A', 65.53, 65.85),
 ('CC', 47.15, 46.27),
 ('CCI', 100.92, 100.77),
 ('CCJ', 10.11, 10.15),
 ('CCU', 26.21, 26.29),
 ('CELG', 135.9, 135.57),
 ('CEMP', 3.85, 3.65),
 ('CL', 71.6, 71.57),
 ('CMCSA', 39.7, 40.63),
 ('CMG', 345.36, 334.61),
 ('CNC', 83.19, 82.73),
 ('COP', 44.91, 45.65),
 ('COR', 109.06, 1

<h1>SANDBOX ---</h1>

In [13]:
FILENAME_INPUT=""
with open(FILENAME_INPUT) as fp:
    soup = BeautifulSoup(fp, "html5lib")

IOError: [Errno 2] No such file or directory: ''

In [ ]:
with open("www/output.html") as fp:
    soup = BeautifulSoup(fp, "html5lib")

In [ ]:

soup.find_all('span')[0].string = "regular: "+str(123)

soup.find_all('span')[0].string

In [ ]:
new_tag = soup.new_tag('li')

a = soup.new_tag('a')
a_content = anomaly[0]+' '
a.string=unicode(string_content)

p = soup.new_tag('p')
p_content = 'Bloomberg:'+str(anomaly[1])+' Google: '+str(anomaly[2])
p['style']="font-size:9px"
p.string=unicode(p_content)
a.insert(1,p)




new_tag.append(a)
     

In [ ]:
new_tag

In [ ]:
a

In [ ]:

soup.find_all('a')[1].string = "Instruments found from the source "+str(comparisons)+"/"+str(dataset)

In [ ]:
soup.find_all('a')[1]

In [ ]:
update_html()

In [ ]:
soup.body.find_all('div')[1]

In [ ]:
soup.body.find('ul')

In [ ]:
string = "PX US Equity|0|10|08/02/2017||0.850000|1.200000||1.200000|1.200000|0.850000|USD|"

In [ ]:
len(string.split(" "))

In [ ]:
def caa():
    a = 123,122,22
    b =2
    return a,b

In [ ]:
a,b = caa()

In [ ]:
a

In [ ]:
Prezzi040817.out20170804

In [ ]:

datetime.datetime.year.today